<a href="https://colab.research.google.com/github/este7734/Web_scraping_project/blob/master/Welt_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='orange'> Welt Web Scraper </font>

---



## Import Dependencies

In [2]:
# Import libraries for processing web text
from bs4 import BeautifulSoup
import requests

from textblob import TextBlob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lxml import html

# Import these dependencies if using Google Colab 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Define All Functions

In [3]:
#@title Functions Hidden
# Get content of the webpage in an html string format by passing a url 
def get_html(url):
    page = requests.get(url)
    html_out = html.fromstring(page.content)
    text = page.text
    return html_out, text

# Convert html into soup to enable soup menthods
def get_soup(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    return soup

# Extract hyperlinks from soup
def get_soup_links(soup):
    links = []
    for link in soup.find_all('a'):
        out_link = link.get('href')
        links.append(out_link)
    return links

# This function is for use with only the Topic pages on reuters.com
# Search through ALL links and filter for only those that are for actual articles
# links are formatted differently 
def get_articles_reuters_topics(links, old_url_set):
    articles = []
    for link in links:
        try:
            if 'https://www.welt.de' not in link:
                if 'article' in link:
                  link = 'https://www.welt.de' + link
                  if url_check(old_url_set, link) == False:
                    articles.append(link)
        except:
            continue
    articles = list(set(articles))
    old_url_set = set(articles + list(old_url_set))       
    return articles, old_url_set

# Check if new urls exists in the old_url_set. if yes, return True; if no, return False
# This function is used in the get_articles_reuters_topics function
def url_check(old_url_set, url):
    url_set = set([url])
    test_set = old_url_set & url_set
    if len(test_set) == 0:
        check = False
    else:
        check = True
    return check

# Get html strings from list of article weblinks
def get_html_reuters(articles):
    soup_list = []
    for article in articles:
        _, text = get_html(article)
        soup = get_soup(text)
        soup_list.append(soup)
    return soup_list

# Break out article_body, article_headline, and article_date from each article in provided hyperlinks and put into a dictionary called: out_list
def get_reuters_elements(soup_list, articles):
    out_list = []
    i = 0
    for article in soup_list:
        link = articles[i] # I don't think this is used at all here, which means there is no reason to require the second argument: articles
        i += 1
        try:
            article_body = article.find_all(body_class, {'class': body_tag})
            article_p = []
            for item in article_body:
                p_list = item.find_all('p')
                for p in p_list:
                    article_p.append(p.text)
            out_text = ' '.join(article_p)
            if out_text == '':
              continue
            if out_text.startswith('Besondere Reportagen'):
              continue
            out_dict = dict([('Text',out_text),('url',link)])
            out_list.append(out_dict)
        except:
            print('Unable to decode...skipping article...')
            continue

    return out_list

## Define URL Variables and Run Functions

<font color='orange'>Step 1.</font> Instantiate `tags and values`. Then instntiate `old_url_set` to be used in the `get_articles_reuters_topics` function. This is a running log of article links that will be compiled by iterating from steps 2 - 3.

In [4]:
# Welt classes and tags
body_class = 'div'
headline_class = 'h2'
date_class = 'div'

body_tag = 'c-sticky-container'
headline_tag = 'c-headline o-dreifaltigkeit__headline rf-o-headline'
date_tag = 'c-container c-container--is-stacked'

# Instantiate empty set to use a running list of hyperlinks while 
# running the scrape iterations
old_url_set = set([])

## Scrape Reuters Topics pages for all the most recent news articles. <font color='orange'>*Run Steps 2 - 3 for each instance of `url` variable, before moving on to the next steps*</font>

<font color='orange'>Step 2.</font> Define variables for each of Reuters main topics pages. Run this cell for each iteration by uncommenting a different url each time.

In [19]:
# Define url variables
# NOTE: You must run these individually through the end of this section
# I didn't have time to figure out how to loop through all of them properly
# There is a section at the very bottom where you can see that I attempted but ran
# into a problem on one of the last functions. 

# Welt Links
#url = r'https://www.welt.de/'
#url = r'https://www.welt.de/politik/'
#url = r'https://www.welt.de/wirtschaft/'
#url = r'https://www.welt.de/vermischtes/'
#url = r'https://www.welt.de/debatte/'
#url = r'https://www.welt.de/wissenschaft/'
#url = r'https://www.welt.de/geschichte/'
#url = r'https://www.welt.de/sonderthemen/'

## Scraper

<font color='orange'>Step 3.</font> Get HTML srting from web `url`

In [20]:
# Pass the each instance of `url` variable to return the web page in HTML format and convert it to a string
html_string = str(get_html(url))
# Pass the HTML string (of the web page) to get its soup
soup = get_soup(html_string)
# Find ALL links on within the soup
links = get_soup_links(soup)
# Use this for Topics Pages only
# Filter out only those links that are for actual articles. We only want the "good" links
# This filters out things like links to images and advertisements or non-news worthy pages
articles, old_url_set = get_articles_reuters_topics(links, old_url_set)
print(len(articles))
# Print out the running list of hyperlinks to see how many you have
print(len(old_url_set))

30
299


In [22]:
#@title Double-click for entire old_url_set
old_url_set = {'https://www.welt.de/Advertorials/deutsche-post-dhl/article206336287/Deutsche-Post-DHL-So-funktioniert-die-Post-von-morgen.html',
 'https://www.welt.de/Advertorials/ferienregion-ortlergebiet/article210058295/Der-groesste-Suedtiroler.html',
 'https://www.welt.de/Advertorials/geero/article207823039/Radfahren-mit-Geero-Mit-Leichtigkeit-in-den-Sommer-starten.html',
 'https://www.welt.de/apps/article118565353/WELT-Edition.html',
 'https://www.welt.de/apps/article174576784/Nutzen-Sie-die-WELT-News-App-auf-Ihrem-Smartphone.html',
 'https://www.welt.de/debatte/article210247285/Medienschelte-Nicht-viel-von-diesem-Drosten-Vorwurf-hat-Bestand.html',
 'https://www.welt.de/debatte/kolumnen/zippert_zappt/article210192303/Zippert-zappt-Schweinefleisch-hat-viele-Gesichter.html',
 'https://www.welt.de/debatte/kolumnen/zippert_zappt/article210310795/Zippert-zappt-Versmold-der-Fettfleck-Deutschlands.html',
 'https://www.welt.de/debatte/kolumnen/zippert_zappt/article210485081/Zippert-zappt-Tunnel-sollten-immer-zwei-Oeffnungen-haben.html',
 'https://www.welt.de/debatte/kommentare/article209801329/Friedensprozess-Weil-Europa-streitet-regiert-in-Libyen-weiter-die-Gewalt.html',
 'https://www.welt.de/debatte/kommentare/article210036225/Mobilitaet-Eine-autofreie-Welt-waere-ein-Albtraum.html',
 'https://www.welt.de/debatte/kommentare/article210088793/Corona-und-die-Fleischindustrie-Toennies-muss-haften.html',
 'https://www.welt.de/debatte/kommentare/article210099459/Brexit-Das-hohle-System-von-Boris-Johnson.html',
 'https://www.welt.de/debatte/kommentare/article210123239/Zippert-zappt-Der-Radfahrer-ist-der-irrationalste-Verkehrsteilnehmer-Vor-dem-Pferd.html',
 'https://www.welt.de/debatte/kommentare/article210153475/Corona-in-Guetersloh-Chef-Lockerer-Laschet-hat-an-Statur-verloren.html',
 'https://www.welt.de/debatte/kommentare/article210240447/Atomwaffen-Wie-Deutschland-sich-zur-Zielscheibe-eines-Angriffs-macht.html',
 'https://www.welt.de/debatte/kommentare/article210313779/Corona-Mit-der-Nachlaessigkeit-kommt-die-zweite-Welle.html',
 'https://www.welt.de/debatte/kommentare/article210314821/Kritik-an-Armin-Laschet-Wer-Kanzler-werden-will-muss-da-durch.html',
 'https://www.welt.de/debatte/kommentare/article210337757/USA-Donald-Trump-ist-vier-Monate-vor-der-Wahl-am-Tiefpunkt.html',
 'https://www.welt.de/debatte/kommentare/article210344793/Die-CDU-ist-75-und-braucht-eine-weltliche-Daseinsbegruendung.html',
 'https://www.welt.de/debatte/kommentare/article210375109/Guetersloh-Laender-muessen-gemeinsame-Regeln-aufstellen.html',
 'https://www.welt.de/debatte/kommentare/article210399541/Deutschland-ein-Albtraum-Nein.html',
 'https://www.welt.de/debatte/kommentare/article210430469/Erdogan-und-Menschenrechte-Die-Verfolgung-kritischer-Stimmen-ist-in-der-Tuerkei-Routine.html',
 'https://www.welt.de/debatte/kommentare/article210442205/Verfassungsdiskussion-Der-Begriff-Rasse-muss-aus-dem-Grundgesetz-gestrichen-werden.html',
 'https://www.welt.de/debatte/kommentare/article210477705/Fussball-und-Corona-Es-lebe-die-Bundesliga.html',
 'https://www.welt.de/debatte/kommentare/article210488111/Tierwohl-und-Verbraucher-Das-Fukushima-der-Fleischindustrie.html',
 'https://www.welt.de/food/essen/article198125595/Vegetarisch-Grillen-Rezepte-mit-Kaese-und-Gemuese-ohne-Fleisch-Tofu.html',
 'https://www.welt.de/food/essen/article210071501/Rezept-Nicht-nur-zum-Grillen-passt-dieses-Gericht.html',
 'https://www.welt.de/food/trinken/article206456499/Nur-bei-WELT-Exklusives-Rotwein-Paket-fuer-29-90-statt-55-70-lieferkostenfrei.html',
 'https://www.welt.de/geschichte/article192520725/Duell-der-Mignons-1578-Frankreichs-beruehmtestes-Duell-endete-im-Blutbad.html',
 'https://www.welt.de/geschichte/article194914723/Versailler-Vertrag-1919-Die-Sieger-kannten-die-Folgen-der-Reparationen.html',
 'https://www.welt.de/geschichte/article196419389/Spiele-im-Kolosseum-Die-Roemer-metzelten-Hunderte-wilde-Tiere-nieder.html',
 'https://www.welt.de/geschichte/article199067983/Cholera-Epidemie-1866-Der-Tod-begleitete-die-preussische-Armee-vor-Wien.html',
 'https://www.welt.de/geschichte/article200553240/Schlacht-von-Poitiers-1356-Englische-Langbogen-gegen-Frankreichs-Ritter.html',
 'https://www.welt.de/geschichte/article209302497/Draft-Riots-New-York-Schwarze-wurden-gelyncht-verbrannt-verstuemmelt.html',
 'https://www.welt.de/geschichte/article209303929/Schiffskatastrophe-Als-SOS-zum-ersten-Mal-Hunderte-Menschen-rettete.html',
 'https://www.welt.de/geschichte/article209351495/Luftbildarchaeologie-So-besetzten-die-Legionen-Britannien.html',
 'https://www.welt.de/geschichte/article209376983/Glienicker-Bruecke-Der-groesste-Agentenaustausch-des-Kalten-Krieges.html',
 'https://www.welt.de/geschichte/article209383237/Die-grosse-Hure-Weinten-die-Juden-wirklich-an-den-Wassern-von-Babylon.html',
 'https://www.welt.de/geschichte/article209404193/Sex-in-Pompeji-Sie-hat-s-hier-mit-2000-netten-Maennern-getrieben.html',
 'https://www.welt.de/geschichte/article209434101/Rassenunruhen-Weisser-Lynchmob-bombardierte-Tulsa-mit-Terpentinbaellen.html',
 'https://www.welt.de/geschichte/article209439603/Gegen-Jack-Sharkey-1930-Schlag-in-Unterleib-macht-Max-Schmeling-zum-Champion.html',
 'https://www.welt.de/geschichte/article209455599/Ludwig-II-von-Bayern-Der-Blick-des-Volkes-besudelt-meine-Schloesser.html',
 'https://www.welt.de/geschichte/article209456885/Marengo-1800-Mit-Tricks-und-Faelschungen-gelang-Napoleons-wichtigster-Sieg.html',
 'https://www.welt.de/geschichte/article209590525/Seuchen-im-Mittelalter-Lepra-Opfern-blieb-nur-noch-Betteln-und-lautes-Klappern.html',
 'https://www.welt.de/geschichte/article209618669/Die-Prohibition-machte-den-Ku-Klux-Klan-zur-Massenbewegung.html',
 'https://www.welt.de/geschichte/article209656005/Todesstrafe-am-Bodensee-Sie-wurden-an-einem-Haken-aufgehaengt.html',
 'https://www.welt.de/geschichte/article209719369/Legenden-An-der-Freien-Republik-Schwarzenberg-stimmt-fast-nichts.html',
 'https://www.welt.de/geschichte/article209753261/Historikerkommission-Wie-braun-war-das-Landwirtschaftsministerium.html',
 'https://www.welt.de/geschichte/article209858329/1870-71-Preussens-Truempfe-im-Krieg-gegen-Frankreich.html',
 'https://www.welt.de/geschichte/article209924401/Nord-Ostsee-Kanal-Seine-Tiefe-bestimmte-die-Groesse-der-deutschen-Schlachtschiffe.html',
 'https://www.welt.de/geschichte/article209925035/Rom-Perser-nutzte-den-Kaiser-als-Fusstritt-und-haeutete-ihn.html',
 'https://www.welt.de/geschichte/article210042989/Abriss-1990-Millionengrab-Berliner-Mauer.html',
 'https://www.welt.de/geschichte/article210210891/Hinrichtungen-Erst-Sex-mit-dem-Chef-dann-Folter-und-Enthauptung.html',
 'https://www.welt.de/geschichte/article210297379/Nordkorea-gegen-Suedkorea-Die-erste-heisse-Schlacht-im-Kalten-Krieg.html',
 'https://www.welt.de/geschichte/article210297763/Steinmeier-Villa-Foto-von-Vorbesitzer-Heymann-aufgetaucht.html',
 'https://www.welt.de/geschichte/article210374219/Goldene-Zwanziger-Die-Weimarer-Republik-voll-auf-Koks.html',
 'https://www.welt.de/geschichte/article210484015/Verschwoerungen-Wurde-Flug-Itavia-870-abgeschossen.html',
 'https://www.welt.de/geschichte/raf/article209923765/Linksterrorismus-Extra-Drill-fuer-verwoehnte-Buergerkinder.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article193675009/Invasion-1944-Nazi-Deutschland-liess-sich-von-den-Fake-Armeen-taeuschen.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article194001949/Hitlers-Plan-1939-Bei-erster-Gelegenheit-Polen-angreifen.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article197416213/Operation-Cobra-Normandie-1944-4700-Tonnen-Bomben-ebneten-den-Weg.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article208908219/Kriegsende-1945-Wohin-die-Rote-Armee-in-Deutschland-verschwand.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article208985201/Auktion-Wehrmachts-Chiffriermaschine-Enigma-erzielt-Sensationspreis.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article209239869/Paris-1940-Die-schlimmste-Niederlage-der-franzoesischen-Geschichte.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article209456313/Hakenkreuze-ueber-Paris-An-der-Uebermacht-der-Sieger-sollte-kein-Zweifel-bestehen.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article209793395/Churchill-Sieg-um-jeden-Preis-Ohne-den-Sieg-gibt-es-kein-Ueberleben.html',
 'https://www.welt.de/geschichte/zweiter-weltkrieg/article210108057/Raetsel-Wann-genau-war-Hitler-denn-nun-in-Paris.html',
 'https://www.welt.de/gesundheit/article196855895/Muecken-und-Insektenstiche-Bite-Away-Kuehlpads-Co-Das-hilft.html',
 'https://www.welt.de/gesundheit/article210205029/Covid-19-Warum-Maenner-haeufiger-sterben.html',
 'https://www.welt.de/gesundheit/psychologie/article148606106/Wenn-Jugendliche-ihren-Koerper-zum-Panzer-staehlen.html',
 'https://www.welt.de/icon/article209118383/Horoskop-So-wird-das-Wochenende-am-27-und-28-Juni-2020.html',
 'https://www.welt.de/icon/partnerschaft/article210157775/Bertelsmann-Studie-Warum-kostet-es-ein-Vermoegen-Mutter-zu-sein.html',
 'https://www.welt.de/kaufberatung/haushalt/article209810629/Raeume-kuehlen-Klimaanlage-Co-Tipps-gegen-Hitze-in-der-Wohnung.html',
 'https://www.welt.de/kmpkt/article207802377/Deutsch-Quiz-10-Fragen-zur-deutschen-Grammatik.html',
 'https://www.welt.de/kmpkt/article208147565/Traeume-werden-von-Umgebungsgeraeuschen-beeinflusst.html',
 'https://www.welt.de/kmpkt/article209177169/Hunde-zeigen-ihren-Besitzern-gegenueber-einen-Beschuetzerinstinkt.html',
 'https://www.welt.de/kultur/article210437427/Urteil-gegen-Kirill-Serebrennikow-Der-russische-Staat-nimmt-Rache-an-einem-Kuenstler.html',
 'https://www.welt.de/kultur/article210487535/Von-extern-Online-Voting-fuer-Wunsch-Tatort-manipuliert-so-reagiert-die-ARD.html',
 'https://www.welt.de/marktplatz/article132865764/Einfach-Kunst-kaufen.html',
 'https://www.welt.de/mediathek/article163230255/So-empfangen-Sie-WELT-und-N24-Doku-in-Full-HD-Qualitaet.html',
 'https://www.welt.de/newsticker/dpa_nt/afxline/topthemen/article210494635/Trump-sagt-Golf-Wochenende-wegen-Anarchisten-ab.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210245911/Otti-Fischer-hat-seine-Lebensgefaehrtin-geheiratet.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210259943/TV-Dreharbeiten-in-Corona-Zeiten.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210291497/Axel-Prahl-hat-geplante-Pause-einfach-vorgezogen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210324965/Dennis-Quaid-heiratet-zum-vierten-Mal.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210325301/Max-Giesinger-Ich-war-schuechtern-und-still.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210326399/NDW-Star-Markus-heiratet-live-im-Fernsehen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210341497/Jennifer-Aniston-hat-unter-Rachel-Green-gelitten.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210398645/Koenig-Felipe-eilt-zur-Rettung-Mallorcas.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210409147/Louvre-oeffnet-wieder-Im-Zickzack-zur-Mona-Lisa.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210415449/Drosten-Podcast-gewinnt-doppelt-beim-Grimme-Online-Award.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210422435/Dixie-Chicks-aendern-ihren-Namen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210426977/Heiko-und-Roman-Lochmann-ueber-ihre-Plaene-als-Musiker.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210436081/Regisseur-Kirill-Serebrennikow-bleibt-in-Freiheit.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210439787/Neil-Youngs-Archiv-Trueffel-sind-Homegrown.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210456655/Vor-25-Jahren-verhuellte-Christo-den-Reichstag.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210457255/Nolans-Action-Thriller-Tenet-erneut-verschoben.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210462329/Wenn-im-Zoo-eine-Affenhitze-herrscht.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210474335/Bob-Dylan-erobert-die-Album-Charts.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210475219/Rassismus-Debatte-fuehrt-zum-Umdenken.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/boulevard_nt/article210492473/Justin-Bieber-reicht-Verleumdungsklage-ein.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/brennpunkte_nt/article209799791/Studie-Blutgruppe-beeinflusst-Schwere-von-Covid-19-Verlauf.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/brennpunkte_nt/article210366417/Coronavirus-EU-Behoerde-empfiehlt-Einsatz-von-Remdesivir.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/brennpunkte_nt/article210494327/EU-Einreisebeschraenkungen-bleiben-fuer-viele-Laender-bestehen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/brennpunkte_nt/article210494617/Motivsuche-nach-Messerattacke-in-Glasgow.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210361281/Getreideernte-nur-unter-Durchschnitt.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210368583/Auskunftsrecht-bei-Streit-ums-Geld-erweitert.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210425323/Corona-Krise-bringt-Nike-tief-in-die-roten-Zahlen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210425631/Deutsche-Bank-meistert-US-Stresstest.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210426257/Ministerin-Kloeckner-will-Fleischmarkt-umkrempeln.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210426639/Mehrwertsteuersenkung-heizt-Preiskampf-bei-Lebensmitteln-an.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210426993/Vonovia-steigt-in-niederlaendischen-Immobilienmarkt-ein.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210427525/Deutlich-mehr-Mineralwasser-produziert.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210428051/KLM-bekommt-staatlichen-Milliardenkredit.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210429425/Wirecard-Philippinen-vermuten-Ex-Vorstand-in-China.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210431779/Dax-schliesst-im-Minus-Anlegerflucht-wegen-Corona-Sorgen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210434359/Corona-Krise-treibt-Modekette-H-M-in-die-roten-Zahlen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210453989/Berlin-im-Wirecard-Skandal-unter-Druck-aus-Bruessel.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210457405/RWE-will-keinen-Strom-aus-Datteln-4.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/article210480611/Wehmut-bei-VW-Werkern-Der-letzte-Verbrenner-aus-Zwickau.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/boersefrankfurttabelle_nt/article210422559/DAX-Schlusskurse-im-Spaethandel-am-25-06-2020-um-20-30-Uhr.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/boersefrankfurttabelle_nt/article210448997/DAX-Kurse-im-XETRA-Handel-am-26-06-2020-um-13-05-Uhr.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/boersefrankfurttabelle_nt/article210485277/DAX-Schlusskurse-im-XETRA-Handel-am-26-06-2020-um-17-55-Uhr.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wirtschaft_nt/boersefrankfurttabelle_nt/article210492577/DAX-Schlusskurse-im-Spaethandel-am-26-06-2020-um-20-30-Uhr.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article209726417/Neuer-Nasa-Rover-startet-im-Juli-zum-Mars.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article209796455/Mehr-Infektionen-mit-Vibrionen-erwartet.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article209905575/Vermutung-West-Nil-Virus-etabliert-sich-in-Deutschland.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article209931491/Aeltere-Kinder-haben-haeufiger-Antikoerper-gegen-Sars-CoV-2.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article209991907/Irland-vor-5000-Jahren-Monumentalgraeber-und-Inzest-Dynastie.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210057409/Klimaschutz-Wissenschaftler-fordern-allgemeinen-CO2-Preis.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210088209/Weitere-Kreisstruktur-nahe-Stonehenge-entdeckt.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210127571/Forscher-wollen-Lockdown-Auswirkungen-auf-Tiere-untersuchen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210214233/Raetselhaftes-Heringssterben-an-der-Nordseekueste.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210214239/Ueberwachungssysteme-scannen-Weltraum-nach-Asteroiden.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210324839/Das-Bett-teilende-Paare-schlafen-besser.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210354419/EU-Entschlossene-Massnahmen-zur-Rettung-der-Meere-noetig.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210355069/Antikoerper-Studie-Viele-Buerger-Ischgls-waren-infiziert.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210355389/Studie-zeigt-dramatischen-Gletscherschwund-in-den-Alpen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210426207/Studie-Baeume-pflegen-statt-pflanzen.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210426225/Cannabis-kann-Familien-zerstoeren.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210433723/Schwedens-Staatsepidemiologe-wirft-WHO-Fehldeutung-vor.html',
 'https://www.welt.de/newsticker/dpa_nt/infoline_nt/wissenschaft_nt/article210457259/Oxford-Forscherin-Gefahr-trotz-Corona-App-nicht-vorbei.html',
 'https://www.welt.de/podcasts/deffner-und-zschaepitz/article210179497/Podcast-Kann-Porsche-Deutschlands-Elektro-Ehre-retten.html',
 'https://www.welt.de/podcasts/die-macher/article210144485/Podcast-Wie-setzen-Sie-sich-als-Frau-in-Maennerbranche-durch-Frau-Wohlfahrt.html',
 'https://www.welt.de/podcasts/gegen-den-corona-koller/article209841469/Podcast-Die-Suche-nach-dem-Corona-Impfstoff-ist-ein-ethischer-Spagat.html',
 'https://www.welt.de/podcasts/gegen-den-corona-koller/article210218939/Podcast-Homeoffice-und-Digitalisierung-nach-drei-Monaten-Corona.html',
 'https://www.welt.de/podcasts/unsere-kleine-welt/article209885589/Podcast-Nach-der-Corona-Krise-mal-nichts-tun-urlaubsreif.html',
 'https://www.welt.de/politik/article210457287/Coronavirus-Merkel-dankt-Bevoelkerung-von-Guetersloh.html',
 'https://www.welt.de/politik/ausland/article210361545/Migration-EU-Kommission-will-Vorschlaege-fuer-Dublin-Reform-verschieben.html',
 'https://www.welt.de/politik/ausland/article210425909/US-Praesident-Donald-Trump-droht-Statuen-Stuerzern-mit-Vergeltung.html',
 'https://www.welt.de/politik/ausland/article210436207/Veruntreuung-Schuldspruch-fuer-russischen-Starregisseur-Serebrennikow.html',
 'https://www.welt.de/politik/ausland/article210446011/Coronavirus-USA-Wie-Trump-den-Rekord-bei-Neuinfektionen-kleinredet.html',
 'https://www.welt.de/politik/ausland/article210459251/Demo-von-Extinction-Rebellion-auf-dem-Rollfeld-des-Flughafens-Orly.html',
 'https://www.welt.de/politik/ausland/article210465819/EU-fehlt-weiter-der-Ueberblick-ueber-gewaltbereiten-Rechtsextremismus.html',
 'https://www.welt.de/politik/ausland/article210481871/Merkel-Muessen-ueber-Verhaeltnis-zu-USA-grundsaetzlich-nachdenken.html',
 'https://www.welt.de/politik/ausland/article210488001/USA-verhaengen-Visa-Restriktionen-gegen-chinesische-Funktionaere.html',
 'https://www.welt.de/politik/ausland/article210492885/USA-Repraesentantenhaus-will-Hauptstadt-Washington-zu-Bundesstaat-machen.html',
 'https://www.welt.de/politik/deutschland/article210345941/Corona-in-Guetersloh-Buergermeister-wuenscht-sich-Unterstuetzung-von-Merkel.html',
 'https://www.welt.de/politik/deutschland/article210346545/Erstmals-wieder-Anstieg-von-Asylantraegen-in-Europa-Deutschland-als-Ausnahme.html',
 'https://www.welt.de/politik/deutschland/article210359589/13-Millionen-Nutzer-Deutschlands-Corona-Warn-App-verschickt-erste-Warnungen.html',
 'https://www.welt.de/politik/deutschland/article210364431/Corona-Ausbruch-in-Warendorf-Das-groesste-Risiko-ist-dass-die-Quarantaene-durchbrochen-wird.html',
 'https://www.welt.de/politik/deutschland/article210371955/CDU-umgarnt-jetzt-Schwule-und-Lesben.html',
 'https://www.welt.de/politik/deutschland/article210384593/Genitalverstuemmelungen-Kein-Wort-das-den-Schmerz-beschreiben-kann.html',
 'https://www.welt.de/politik/deutschland/article210397131/AfD-Ist-Andreas-Kalbitz-Mitglied-oder-nicht-Oder-beides.html',
 'https://www.welt.de/politik/deutschland/article210417291/Unternehmen-in-NRW-Die-Angst-in-den-Toennies-Sog-zu-geraten.html',
 'https://www.welt.de/politik/deutschland/article210426387/Corona-Ausbruch-in-NRW-Laschet-prueft-Haftbarkeit-von-Toennies.html',
 'https://www.welt.de/politik/deutschland/article210432777/Stuttgart-Polizei-ermittelt-in-den-eigenen-Reihen-nach-Krawallnacht.html',
 'https://www.welt.de/politik/deutschland/article210436159/Ende-Gelaende-Kohlegegner-besetzen-Bagger-in-Garzweiler-und-Jaenschwalde.html',
 'https://www.welt.de/politik/deutschland/article210440381/ZDF-Politbarometer-Union-steigt-auf-40-Prozent-Merkel-legt-bei-Beliebtheit-weiter-zu.html',
 'https://www.welt.de/politik/deutschland/article210442547/Guetersloh-Arbeiter-wirft-Toennies-Versaeumnisse-bei-der-Corona-Bekaempfung-vor.html',
 'https://www.welt.de/politik/deutschland/article210449497/Baden-Wuerttemberg-Schottergaerten-sollen-per-Gesetz-endgueltig-verschwinden.html',
 'https://www.welt.de/politik/deutschland/article210459531/Gruene-Habeck-und-Baerbock-erklaeren-ihren-Fuehrungsanspruch.html',
 'https://www.welt.de/politik/deutschland/article210474071/Waffen-Sprengstoff-und-Nazi-Literatur-bei-KSK-Soldat-in-Sachsen-gefunden.html',
 'https://www.welt.de/regionales/baden-wuerttemberg/article210494533/Hoffenheim-kaempft-in-Dortmund-um-Europa-League-Direktticket.html',
 'https://www.welt.de/regionales/bayern/article210494513/Bayern-bekommt-Meisterschale-Augsburg-will-das-Maximale.html',
 'https://www.welt.de/regionales/berlin/article210494517/Bussgeld-droht-bei-Verstoessen-gegen-Maskenpflicht.html',
 'https://www.welt.de/regionales/hessen/article210494585/Eintracht-Frankfurt-will-siegreichen-Saisonabschluss.html',
 'https://www.welt.de/regionales/mecklenburg-vorpommern/article210494591/Hansa-Rostock-vor-Endspiel-in-Wuerzburg-zuversichtlich.html',
 'https://www.welt.de/regionales/niedersachsen/article210494553/Werder-Bremen-droht-erster-Bundesliga-Abstieg-seit-40-Jahren.html',
 'https://www.welt.de/regionales/nrw/article210494581/Schalke-Fans-demonstrieren-gegen-Toennies.html',
 'https://www.welt.de/regionales/sachsen/article210494529/Neue-Synode-der-evangelischen-Landeskirche-konstituiert-sich.html',
 'https://www.welt.de/regionales/thueringen/article210494563/Thueringen-will-beim-Erhalt-von-Streuobstwiesen-helfen.html',
 'https://www.welt.de/reise/nah/article210399943/Almschule-Erlebnis-Angebote-fuer-Kinder-im-Salzburger-Land.html',
 'https://www.welt.de/reise/staedtereisen/article209832711/Parfuem-aus-Koeln-Wie-Koelnisch-Wasser-die-Tuerkei-eroberte.html',
 'https://www.welt.de/satire/article210458055/Statt-Fleisch-Toennies-Co-stellen-auf-Billig-Klamotten-um.html',
 'https://www.welt.de/services/article122129231/Nutzungsbedingungen-WELT-Digital.html',
 'https://www.welt.de/services/article157550705/Datenschutzerklaerung-WELT-DIGITAL.html',
 'https://www.welt.de/services/article157911156/Jugendschutz.html',
 'https://www.welt.de/services/article7893735/Impressum.html',
 'https://www.welt.de/services/article7894222/Kontakt.html',
 'https://www.welt.de/sonderthemen/article207886915/Alltag-im-Homeschooling.html',
 'https://www.welt.de/sonderthemen/article207903327/Rechte-und-Pflichten-im-Homeoffice.html',
 'https://www.welt.de/sonderthemen/article207918393/Gesellschaft-Von-Zuhause-arbeiten-heisst-flexibler-arbeiten.html',
 'https://www.welt.de/sonderthemen/article207945927/Kultur-online-waehrend-des-Shutdown.html',
 'https://www.welt.de/sonderthemen/article207951797/Neue-Moebel-fuer-ein-mobiles-Leben.html',
 'https://www.welt.de/sonderthemen/article208073087/Gewinner-und-Verlierer-der-Krise.html',
 'https://www.welt.de/sonderthemen/article208073149/Gastronomie-kaempft-ums-Ueberleben.html',
 'https://www.welt.de/sonderthemen/article208073193/Neue-Unternehmen-in-der-Lausitz.html',
 'https://www.welt.de/sonderthemen/article208073257/Wiederbeginn-nach-der-Corona-Krise.html',
 'https://www.welt.de/sonderthemen/article208079731/Strategien-des-Mittelstandes.html',
 'https://www.welt.de/sonderthemen/beethoven/article205072046/Kuenstliche-Intelligenz-soll-Beethovens-legendaere-zehnte-Sinfonie-vollenden.html',
 'https://www.welt.de/sonderthemen/beethoven/article205075098/Annaeherung-an-das-Genie-Beethoven.html',
 'https://www.welt.de/sonderthemen/beethoven/article205075244/Beethovens-Geburtsstadt-feiert-den-Komponisten-mit-einem-riesigen-Programm.html',
 'https://www.welt.de/sonderthemen/beethoven/article205075442/Beethovens-Kampf-gegen-die-Taubheit.html',
 'https://www.welt.de/sonderthemen/beethoven/article205078792/Beethovens-Leben-und-Werk.html',
 'https://www.welt.de/sonderthemen/beethoven/article205108788/Viele-Plaetze-die-heute-Pilgerstaette-sind-kannte-Beethoven-gar-nicht.html',
 'https://www.welt.de/sonderthemen/beethoven/article205175503/Beethoven-und-die-Damen.html',
 'https://www.welt.de/sonderthemen/beethoven/article205213005/Beethoven-und-politische-Ideale.html',
 'https://www.welt.de/sonderthemen/energiewende/article209522679/Bund-will-EEG-Umlage-bremsen.html',
 'https://www.welt.de/sonderthemen/messewirtschaft/article205564425/Coronavirus-sorgt-fuer-leichten-Rueckgang-von-Ausstellern-und-Besuchern-auf-Messen-in-Deutschland.html',
 'https://www.welt.de/sonderthemen/messewirtschaft/article206281669/Wegen-COVID-19-finden-weltweit-viele-Messen-nicht-statt-oder-werden-verschoben.html',
 'https://www.welt.de/sonderthemen/messewirtschaft/article207072685/Die-Corona-Krise-trifft-Messebauer-schwer-Manche-werden-erfinderisch.html',
 'https://www.welt.de/sonderthemen/messewirtschaft/article207731021/Corona-Krise-laehmt-Messewirtschaft-weiter-Die-Branche-fordert-Entscheidungen-von-der-Politik.html',
 'https://www.welt.de/sonderthemen/messewirtschaft/article208721429/Analog-trifft-Digital.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208692253/Oesterreichs-beste-Rotweine.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208692619/Kulinarik-und-Sehenswuerdigkeiten-im-Weinviertel.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208692895/Entdeckung-der-Wiener-Weinkultur.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208695059/Oesterreichs-Anbaugebiete-fuer-Riesling.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208713819/Austrias-Erfolge-mit-Weissburgunder-und-Chardonnay.html',
 'https://www.welt.de/sonderthemen/weinland-oesterreich/article208718247/Oesterreichs-kulinarische-Vielfalt.html',
 'https://www.welt.de/spiele/online-spiele/article1336655/Sudoku.html',
 'https://www.welt.de/spiele/online-spiele/article8000221/Kreuzwortraetsel-Taeglich-neue-Fragen.html',
 'https://www.welt.de/sport/article210439557/Fussball-Thueringen-hat-erstmals-seit-der-Wende-keinen-Profiklub.html',
 'https://www.welt.de/sport/article210457401/Bernie-Ecclestone-In-vielen-Faellen-sind-Schwarze-rassistischer-als-Weisse.html',
 'https://www.welt.de/sport/fussball/article210343945/Werder-Bremen-Claudio-Pizarro-Traurigleiser-Abgang-einer-Legende.html',
 'https://www.welt.de/sport/fussball/article210430449/FC-Liverpool-Sorry-Gentlemen-Klopp-bricht-Interview-ab-und-geht.html',
 'https://www.welt.de/sport/fussball/article210433721/Presse-ueber-Juergen-Klopp-Eine-Maschine-das-war-eine-Vernichtung.html',
 'https://www.welt.de/sport/fussball/article210485021/BVB-Hakimi-vor-Wechsel-nach-Italien.html',
 'https://www.welt.de/vermischtes/article206504969/Corona-Schwedens-Staatsepidemiologe-Anders-Tegnell-aeussert-erstmals-Selbstkritik.html',
 'https://www.welt.de/vermischtes/article206504969/Coronavirus-Alle-Karten-Zahlen-und-Daten-zur-Ausbreitung.html',
 'https://www.welt.de/vermischtes/article210043349/Michel-Ruge-7-Tipps-um-den-richtigen-Trainer-fuer-Selbstverteidigung-zu-finden.html',
 'https://www.welt.de/vermischtes/article210136605/Spanien-lacht-ueber-Restauration-Diese-Aufwertung-ging-gewaltig-schief.html',
 'https://www.welt.de/vermischtes/article210184659/Niedersachsen-Nun-auch-Corona-Infektionen-in-Wiesenhof-Schlachthof-bei-Oldenburg.html',
 'https://www.welt.de/vermischtes/article210210675/Trump-will-aus-Deutschland-abgezogene-Truppen-in-Polen-stationieren.html',
 'https://www.welt.de/vermischtes/article210213091/Virologe-Christian-Drosten-warnt-vor-zweiter-Corona-Welle.html',
 'https://www.welt.de/vermischtes/article210214247/Trotz-Corona-Hundefleisch-Festival-in-China-eroeffnet-Tierschuetzer-warnen.html',
 'https://www.welt.de/vermischtes/article210214423/Unwetter-in-der-Tuerkei-Hier-tobt-eine-Wasserhose-vor-Istanbul.html',
 'https://www.welt.de/vermischtes/article210214457/Ron-Jeremy-Porno-Star-wegen-mehrfacher-Vergewaltigung-angeklagt.html',
 'https://www.welt.de/vermischtes/article210232793/Stiftung-Warentest-Der-teuerste-Sonnenschutz-fuer-Kinder-faellt-durch.html',
 'https://www.welt.de/vermischtes/article210237757/Sea-Watch-3-Mehrere-Migranten-in-Italien-positiv-auf-Corona-getestet.html',
 'https://www.welt.de/vermischtes/article210264379/Koeln-Hessen-Paris-Acht-Festnahmen-bei-Razzia-gegen-Grossfamilie.html',
 'https://www.welt.de/vermischtes/article210283443/Nach-Kritik-Sachsen-hebt-Mathe-Abiturnote-um-einen-Punkt-an.html',
 'https://www.welt.de/vermischtes/article210323385/USA-Corona-34-700-neue-Faelle-Virus-verbreitet-sich-dramatisch-schnell.html',
 'https://www.welt.de/vermischtes/article210401707/Corona-in-Grossbritannien-Briten-ueberrennen-Straende-Gesundheitsminister-droht-mit-Konsequenzen.html',
 'https://www.welt.de/vermischtes/article210422461/Podcast-mit-Christian-Drosten-und-Rezo-gewinnen-Grimme-Online-Award.html',
 'https://www.welt.de/vermischtes/article210426085/Maybrit-Illner-Erklaerungsversuche-fuer-den-Stuttgarter-Gewaltexzess.html',
 'https://www.welt.de/vermischtes/article210432977/Thueringen-Mann-ueberfaehrt-Polizist-mit-Auto-Beamter-schwer-verletzt.html',
 'https://www.welt.de/vermischtes/article210438193/Robbie-Williams-schockt-mit-Verschwoerungstheorien-Pizzagate-Co.html',
 'https://www.welt.de/vermischtes/article210439025/Schleswig-Holstein-Polizei-verliert-bei-Einsatz-schussbereite-Pistole.html',
 'https://www.welt.de/vermischtes/article210462965/Glasgow-Polizei-stoppt-Messerattacke-Verdaechtiger-tot.html',
 'https://www.welt.de/vermischtes/article210473165/Von-Argentinien-bis-Bolivien-Heuschrecken-Plage-in-Suedamerika.html',
 'https://www.welt.de/vermischtes/article210474961/Mitgliederschwund-Kirchen-beklagen-Rekordzahl-an-Austritten.html',
 'https://www.welt.de/wirtschaft/article210335917/Milliardenschwerer-Skandal-Wirecard-stellt-Insolvenzantrag-Aktie-zeitweise-unter-drei-Euro.html',
 'https://www.welt.de/wirtschaft/article210340859/Wirecard-Erstmals-ein-insolventes-Unternehmen-im-Dax.html',
 'https://www.welt.de/wirtschaft/article210344365/J-D-Power-Tesla-landet-in-US-Qualitaetsstudie-ganz-hinten.html',
 'https://www.welt.de/wirtschaft/article210345687/Rentenerhoehung-2021-koennte-im-Westen-ausfallen.html',
 'https://www.welt.de/wirtschaft/article210358529/Bayers-Monsanto-Deal-wird-immer-teurer-dumm-aber-ist-er-keineswegs.html',
 'https://www.welt.de/wirtschaft/article210374153/Media-and-Games-Invest-Videospiele-boomen-in-der-Krise.html',
 'https://www.welt.de/wirtschaft/article210382671/Lachs-Umweltschuetzer-kritisieren-Zuchtmethoden.html',
 'https://www.welt.de/wirtschaft/article210399433/Untergang-im-Zeitraffer.html',
 'https://www.welt.de/wirtschaft/article210399561/Babyboomer-Paare-wuenschen-sich-oft-gemeinsamen-Renten-Einstieg.html',
 'https://www.welt.de/wirtschaft/article210408579/Hauptversammlung-Lufthansa-Aktionaere-stimmen-Staatshilfe-zu.html',
 'https://www.welt.de/wirtschaft/article210419827/Lufthansa-Rettung-ist-perfekt-Tickets-werden-binnen-sechs-Wochen-erstattet.html',
 'https://www.welt.de/wirtschaft/article210425867/Corona-Stresstest-Fed-verbietet-Banken-vorerst-Aktienrueckkaeufe.html',
 'https://www.welt.de/wirtschaft/article210427839/Corona-in-Schlachthoefen-Kritiker-werfen-Kloeckner-Show-Veranstaltung-vor.html',
 'https://www.welt.de/wirtschaft/article210448163/Wirecard-Als-erster-Vertragspartner-aeussert-sich-Aldi-zum-Desaster.html',
 'https://www.welt.de/wirtschaft/article210473345/Grossbritannien-in-der-Krise-Briten-fuerchten-Ausschreitungen-durch-Lockdown-Muedigkeit.html',
 'https://www.welt.de/wirtschaft/article210494589/Facebook-will-fragwuerdige-Trump-Postings-kuenftig-doch-markieren.html',
 'https://www.welt.de/wirtschaft/gruenderszene/article209586855/Corona-hat-gezeigt-dass-Homeoffice-den-Bullshit-eliminiert.html',
 'https://www.welt.de/wirtschaft/gruenderszene/article209884577/Gorillas-Dieser-Lieferdienst-bringt-Lebensmittel-in-unter-10-Minuten.html',
 'https://www.welt.de/wirtschaft/gruenderszene/article209995303/Arbeitsplaetze-Wie-die-Corona-Krise-das-Leben-im-Buero-veraendert.html',
 'https://www.welt.de/wirtschaft/karriere/article201427870/Arbeit-als-Rentner-2-Karriere-im-Ruhestand-So-gelingt-sie.html',
 'https://www.welt.de/wirtschaft/karriere/article207947649/Handwerksberufe-Wo-Frauen-die-freie-Auswahl-haben.html',
 'https://www.welt.de/wirtschaft/karriere/article210332535/Karriereplanung-Warum-Frauen-mutiger-werden-muessen.html',
 'https://www.welt.de/wirtschaft/karriere/article210449507/Arbeitsbelastung-So-viel-Urlaubsanspruch-haben-Sie-in-Teilzeit.html',
 'https://www.welt.de/wirtschaft/webwelt/article194852057/Laptop-fuer-Studenten-Diese-Notebooks-eignen-sich-fuers-Studium.html',
 'https://www.welt.de/wirtschaft/webwelt/article195841795/AirPods-Alternative-Welche-In-Ear-Kopfhoerer-aehneln-dem-Original.html',
 'https://www.welt.de/wirtschaft/webwelt/article196987959/Unterwasserkamera-Kameras-die-sich-zum-Tauchen-Schnorcheln-eignen.html',
 'https://www.welt.de/wirtschaft/webwelt/article203640208/Airpods-laden-Damit-lassen-sich-die-Apple-AirPods-schnell-versogen.html',
 'https://www.welt.de/wirtschaft/webwelt/article205739071/AirPods-Was-liefern-die-AirPods-Pro-im-Vergleich-zu-aelteren-Modellen.html',
 'https://www.welt.de/wirtschaft/webwelt/article210437525/Sebastian-Thrun-im-Interview-Flugtaxis-fruehestens-in-fuenf-Jahren.html',
 'https://www.welt.de/wirtschaft/webwelt/technik-ratgeber/article193634159/Spiegelreflexkamera-fuer-Einsteiger-Welche-ist-fuer-Anfaenger-ideal.html',
 'https://www.welt.de/wissenschaft/article181602834/Stammzellen-Forscher-zuechten-Vorlaeufer-menschlicher-Eizellen-im-Labor.html',
 'https://www.welt.de/wissenschaft/article184979528/Gentest-an-Embryonen-Huerden-durch-Ethikkommision-zu-hoch.html',
 'https://www.welt.de/wissenschaft/article192988061/Riesenloewe-Simbakubwa-Gross-wie-ein-Nashorn-mit-der-Gestalt-eines-Wolfs.html',
 'https://www.welt.de/wissenschaft/article198033885/Von-Teer-und-Maeusen-Das-sind-die-laengsten-Experimente-der-Welt.html',
 'https://www.welt.de/wissenschaft/article198805769/Teleportation-Was-sich-mit-dem-Quanteninternet-alles-aendern-wird.html',
 'https://www.welt.de/wissenschaft/article198949045/Gehoergangsexostose-Das-Problem-Surfer-Ohr-ist-so-alt-wie-die-Menschheit.html',
 'https://www.welt.de/wissenschaft/article209150033/Corona-Studie-Blutgruppe-koennte-Schwere-des-Covid-19-Verlaufs-beeinflussen.html',
 'https://www.welt.de/wissenschaft/article209674487/Coronavirus-Kinder-sind-seltener-mit-Covid-19-infiziert-als-ihre-Eltern.html',
 'https://www.welt.de/wissenschaft/article209684171/Terahertzwellen-Forscher-entdecken-biologische-Wirkung.html',
 'https://www.welt.de/wissenschaft/article209712709/Corona-WHO-feiert-Ergebnisse-zu-Medikament-als-Durchbruch.html',
 'https://www.welt.de/wissenschaft/article209790417/Corona-Studie-Oeffentliche-Toiletten-koennten-zur-Verbreitung-beitragen.html',
 'https://www.welt.de/wissenschaft/article209808259/Universitaetsklinik-Kiel-Neue-Studie-bestaetigt-Blutgruppe-beeinflusst-Covid-19-Verlauf.html',
 'https://www.welt.de/wissenschaft/article209941957/Corona-Forschung-Warum-Wissenschaft-und-Oeffentlichkeit-in-der-Krise-aneinandergeraten.html',
 'https://www.welt.de/wissenschaft/article210148427/Jupitermond-Europa-Suche-nach-Biomolekuelen-im-Eis.html',
 'https://www.welt.de/wissenschaft/article210167603/Polizeigewalt-in-USA-Neue-Strategien-zur-Deeskalation.html',
 'https://www.welt.de/wissenschaft/article210427167/Das-Raetsel-um-die-niedrige-Corona-Sterblichkeit-in-den-Anden.html'}

## <font color='skyblue'>Parse soup from entire list of hyperlinks that you just accumulated</font>

<font color='orange'>Step 4.</font> Get soup for every link

<font color='orange'>Step 5.</font> Parse the soup for each link into `article_body`, `article_title`, and `article_date`. Create list of dictionaries for each web page

<font color='orange'>Step 6.</font> Run `TextBlob` on list of dictionaries to separate all sentences in a single list

In [23]:
# Get soup for each one of the "good" links
url_links = list(old_url_set) # Convert the running set of links to a list for use in the following functions
print(f'Length of url_links: {len(url_links)}')
len(url_links)

soup_list = get_html_reuters(url_links)
print(f'Length of Soup List: {len(soup_list)}')
#print(soup_list[0])

# Parse the soup for each "good" link to get article text, title, and date
out_list = get_reuters_elements(soup_list, url_links)
print(f'Length of out_list: {len(out_list)}')
#out_list[0:2]

# Translate all articles to English
blob_full=[]
for i in range(len(out_list)):
  try:
    blob = out_list[i]['Text']
    trans = TextBlob(blob) # Enter string object
    trans = trans.translate(to='en')
    blob_full.append(trans)
    #print('\n', i+1, '\n',trans)
  except:
    print('got an error ...., skipping article....')
print(f'\nYou have {len(blob_full)} total blobs from {i+1} different articles')

# Break articles into sentences to conduct keyword search
# in next step
blob_sentences = []

for i in range(len(blob_full)):
  for item in blob_full[i].sentences:
    blob_sentences.append(item)
print(f'\nYou have {len(blob_sentences)} sentences from {i} different articles')

Length of url_links: 299
Length of Soup List: 299
Length of out_list: 298

You have 298 total blobs from 298 different articles

You have 11799 sentences from 297 different articles


This is just a troubleshooting section to ensure you're `TextBlob` came out right. You should see a list of stentences, each starting with the word `Sentence`. Check this before moving to the next step.

In [ ]:
#@title Debugging Section
# print the first 6 sentences so you see what it looks like
blob_sentences[0]

## <font color='skyblue'> Keyword Search </font>



<font color='orange'>Step 7.</font> Determine key words used to search for event of interest in all the articles

In [24]:
# Allow user to type in key words to search the text for
# Note this is case sensitive... so you need to make sure you enter your search 
# Try the entering the following to see some results: corona,COVID,death
filter_list = input("Enter key words to search for separated by commas. don't use spaces.\nSearch is case sensitive.\nExample search: coronavirus,COVID,death\n\n") #.title() # This is still a string... not a list yet

Enter key words to search for separated by commas. don't use spaces.
Search is case sensitive.
Example search: coronavirus,COVID,death

America,United States,USA,U.S.A.,US,U.S.


<font color='orange'>Step 8.</font> Convert keywords into an iterable list for use in the next step


In [25]:
# Split filter words and convert into a list for itterating in the next step 

f = []
for word in (filter_list.split(",")):  # Split string into separate words, separate by comma
  f.append(word)                       # Generate new list containing each key word
f # This is now a list of key words that the user typed in

['America', 'United States', 'USA', 'U.S.A.', 'US', 'U.S.']

<font color='orange'>Step 9.</font> <font color='skyblue'> Search </font>  All sentences for Key Words and return only those consisting of a key word



In [26]:
# Instantiate list for holding the sentences
sentences = []

# Generate empty list of lists to store the sentences with your key words in them
for i in range(len(f)):
  sentences.append([])
#print('Here is what you just made, and empty list of lists: ', sentences, '\n')

# Generate lists of sentences for each key word and plug them into the list of lists from above            
for i in range(len(f)):
  for sentence in blob_sentences:
    if f[i] in sentence:
        sentences[i].append(sentence)
        
# Print number of sentences containing each key word
# Print out all sentences containing each key word
for i in range(len(f)):
  print('='*200)   
  print('\nThere are {} sentences containing the word: {} '.format(len(sentences[i]), f[i])) 
  print('-'*200)   
  #for sentence in sentences[i]:
      #print(sentence)

# write resulst to text file
file_welt = open("MyFile_Welt.txt", "w") 
file_welt.write(str(sentences))
file_welt.close()      


There are 88 sentences containing the word: America 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

There are 53 sentences containing the word: United States 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

There are 45 sentences containing the word: USA 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

There are 0 sentences containing the word: U.S.A. 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## <font color='black'>Step 10. Store All article data in DataFrame </font>  
<font color='grey'>Note: This is not just the filtered data, this contains all information from each web page. While not used in this project, it represents all of the original data used in this run. It can be used for reference.</font> 

In [ ]:
# Put parsed data into Pandas DataFrame
pd.DataFrame(out_list)

,Text,url
0,"75 Jahre, so alt wird die CDU. Daher hat Part...",https://www.welt.de/politik/deutschland/articl...
1,Wir freuen uns sehr über Ihr Interesse an WELT...,https://www.welt.de/services/article157550705/...
2,Ein russisches Gericht hat den Starregisseur K...,https://www.welt.de/politik/ausland/article210...
3,Ein wichtiger Grundstein des Digitalisierungsp...,https://www.welt.de/Advertorials/deutsche-post...
4,Bei Fragen und Anregungen hilft Ihnen unser Te...,https://www.welt.de/services/article7894222/Ko...
5,Ein weiterer Corona-Ausbruch in einem Fleisch...,https://www.welt.de/politik/deutschland/articl...
6,Die Stationierung von US-Soldaten in Deutschla...,https://www.welt.de/politik/ausland/article210...
7,Eigentlich wollte Innenminister Horst Seehofe...,https://www.welt.de/politik/ausland/article210...
8,US-Präsident Donald Trump hat den Sturz von St...,https://www.welt.de/politik/ausland/article210...
9,Lassen Sie sich von einzigartigen Kunstwerken ...,https://www.welt.de/marktplatz/article13286576...
